In [1]:
#import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.metrics.distance import edit_distance
nltk.download('stopwords')
import string
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
import emoji
import contractions

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#import dataset
df = pd.read_excel('dataset_excelfix.xlsx')
df.head(10)

,Label,tweet
0,Negatif,@The_BosGenk Ada apa ni ko ada taggar #GoodBy...
1,Negatif,"@The_BosGenk sssst,, #GoodByeJokowi #GoodByeJ..."
2,Negatif,@Android_AK_47 Jangan biarkan Mahasiswa Berger...
3,Negatif,"@akunkelima212 Two period it's been so long, I..."
4,Negatif,"Di era Jokowi NASAKOM dihidupkan kembali, Isla..."
5,Negatif,Hampir tiap hari ada trand di minta mundurr......
6,Positif,@Nicho_Silalahi Satu bangsa satu tanah air sat...
7,Negatif,ðŸ‘¦: Oo... jadi giniii ðŸ‘§: Nah iya gitu Em...
8,Positif,"@handajani10 aamiiin ya rob, , ðŸ¤² mdh""an Ma..."
9,Positif,Agar semua berjalan normal kembali #GoodByeJok...


In [3]:
#Cleaning Text
def cleaning_text(text):
    # replace RT tag
    text = re.sub('RT\s', '', text)
    # replace @_username
    text = re.sub('\B@\w+', '', text)
    # replace emoji dengan text
    text = emoji.demojize(text)
    # replace URL
    text = re.sub('(http|https):\/\/\S+', '', text)
    # replace #_something_
    text = re.sub('#+', '', text)
    # konversi huruf kapital ke huruf kecil semua
    text = text.lower()
    # replace kata yang berulang-ulang ('oooooo' menjadi '00')
    text = re.sub(r'(.)\1+', r'\1\1', text)
    # replace punctuation repetition dengan single occurance ('!!!!!!' ,enjadi '!')
    text = re.sub(r'[\?\.\!]+(?=[\?.\!])', '', text)
    # menhilangkan angka dan spesial karakter, hanya mengambil alfabet saja
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # replace contractions 
    text = contractions.fix(text)
    return text

df['tweet'] = df['tweet'].apply(lambda x: cleaning_text(x))
df.head(10)

,Label,tweet
0,Negatif,ada apa ni ko ada taggar goodbyejokowi
1,Negatif,sst goodbyejokowi goodbyejokowi
2,Negatif,jangan biarkan mahasiswa bergerak sendirian ...
3,Negatif,two period it s been so long i can t wait an...
4,Negatif,di era jokowi nasakom dihidupkan kembali isla...
5,Negatif,hampir tiap hari ada trand di minta mundurr t...
6,Positif,satu bangsa satu tanah air satu kata goodbyej...
7,Negatif,oo jadi ginii nah iya gitu emote...
8,Positif,aamiin ya rob mdh an mak trade...
9,Positif,agar semua berjalan normal kembali goodbyejoko...


In [4]:
#Tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tokenization'] = df['tweet'].apply(lambda x: tokenization(x.lower()))
df.head(10)

,Label,tweet,Tokenization
0,Negatif,ada apa ni ko ada taggar goodbyejokowi,"[, ada, apa, ni, ko, ada, taggar, goodbyejokowi]"
1,Negatif,sst goodbyejokowi goodbyejokowi,"[, sst, goodbyejokowi, goodbyejokowi]"
2,Negatif,jangan biarkan mahasiswa bergerak sendirian ...,"[, jangan, biarkan, mahasiswa, bergerak, sendi..."
3,Negatif,two period it s been so long i can t wait an...,"[, two, period, it, s, been, so, long, i, can,..."
4,Negatif,di era jokowi nasakom dihidupkan kembali isla...,"[di, era, jokowi, nasakom, dihidupkan, kembali..."
5,Negatif,hampir tiap hari ada trand di minta mundurr t...,"[hampir, tiap, hari, ada, trand, di, minta, mu..."
6,Positif,satu bangsa satu tanah air satu kata goodbyej...,"[, satu, bangsa, satu, tanah, air, satu, kata,..."
7,Negatif,oo jadi ginii nah iya gitu emote...,"[, oo, jadi, ginii, nah, iya, gitu, emote, bat..."
8,Positif,aamiin ya rob mdh an mak trade...,"[, aamiin, ya, rob, mdh, an, mak, trade, mark,..."
9,Positif,agar semua berjalan normal kembali goodbyejoko...,"[agar, semua, berjalan, normal, kembali, goodb..."


In [23]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
kalimat = "ada siapa di mana hehe gagag tata"

tokens = nltk.tokenize.word_tokenize(kalimat)
kemunculan = nltk.FreqDist(tokens)
print(kemunculan.most_common())

[('ada', 1), ('siapa', 1), ('di', 1), ('mana', 1), ('hehe', 1), ('gagag', 1), ('tata', 1)]


In [14]:
#Stop Removal

stopword = nltk.corpus.stopwords.words('indonesian')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

df['Stop_Removal'] = df['Tokenization'].apply(lambda x: remove_stopwords(x))
df.head(10)

,Label,tweet,Tokenization,Stop_Removal
0,Negatif,ada apa ni ko ada taggar goodbyejokowi,"[, ada, apa, ni, ko, ada, taggar, goodbyejokowi]","[, ni, ko, taggar, goodbyejokowi]"
1,Negatif,sst goodbyejokowi goodbyejokowi,"[, sst, goodbyejokowi, goodbyejokowi]","[, sst, goodbyejokowi, goodbyejokowi]"
2,Negatif,jangan biarkan mahasiswa bergerak sendirian ...,"[, jangan, biarkan, mahasiswa, bergerak, sendi...","[, biarkan, mahasiswa, bergerak, rakyat, buruh..."
3,Negatif,two period it s been so long i can t wait an...,"[, two, period, it, s, been, so, long, i, can,...","[, two, period, it, s, been, so, long, i, can,..."
4,Negatif,di era jokowi nasakom dihidupkan kembali isla...,"[di, era, jokowi, nasakom, dihidupkan, kembali...","[era, jokowi, nasakom, dihidupkan, islam, umma..."
5,Negatif,hampir tiap hari ada trand di minta mundurr t...,"[hampir, tiap, hari, ada, trand, di, minta, mu...","[trand, mundurr, ngak, dihiraukan, tdk, pura, ..."
6,Positif,satu bangsa satu tanah air satu kata goodbyej...,"[, satu, bangsa, satu, tanah, air, satu, kata,...","[, bangsa, tanah, air, goodbyejokowi]"
7,Negatif,oo jadi ginii nah iya gitu emote batu man...,"[, oo, jadi, ginii, nah, iya, gitu, emote, bat...","[, oo, ginii, iya, gitu, emote, batu, nii, goo..."
8,Positif,aamiin ya rob mdh an mak trade mark goodby...,"[, aamiin, ya, rob, mdh, an, mak, trade, mark,...","[, aamiin, ya, rob, mdh, an, mak, trade, mark,..."
9,Positif,agar semua berjalan normal kembali goodbyejoko...,"[agar, semua, berjalan, normal, kembali, goodb...","[berjalan, normal, goodbyejokowi, goodbyejokowi]"


In [15]:
#Case Folding
df['tweet'] = df['tweet'].str.lower()
df.head(10)

,Label,tweet,Tokenization,Stop_Removal
0,Negatif,ada apa ni ko ada taggar goodbyejokowi,"[, ada, apa, ni, ko, ada, taggar, goodbyejokowi]","[, ni, ko, taggar, goodbyejokowi]"
1,Negatif,sst goodbyejokowi goodbyejokowi,"[, sst, goodbyejokowi, goodbyejokowi]","[, sst, goodbyejokowi, goodbyejokowi]"
2,Negatif,jangan biarkan mahasiswa bergerak sendirian ...,"[, jangan, biarkan, mahasiswa, bergerak, sendi...","[, biarkan, mahasiswa, bergerak, rakyat, buruh..."
3,Negatif,two period it s been so long i can t wait an...,"[, two, period, it, s, been, so, long, i, can,...","[, two, period, it, s, been, so, long, i, can,..."
4,Negatif,di era jokowi nasakom dihidupkan kembali isla...,"[di, era, jokowi, nasakom, dihidupkan, kembali...","[era, jokowi, nasakom, dihidupkan, islam, umma..."
5,Negatif,hampir tiap hari ada trand di minta mundurr t...,"[hampir, tiap, hari, ada, trand, di, minta, mu...","[trand, mundurr, ngak, dihiraukan, tdk, pura, ..."
6,Positif,satu bangsa satu tanah air satu kata goodbyej...,"[, satu, bangsa, satu, tanah, air, satu, kata,...","[, bangsa, tanah, air, goodbyejokowi]"
7,Negatif,oo jadi ginii nah iya gitu emote batu man...,"[, oo, jadi, ginii, nah, iya, gitu, emote, bat...","[, oo, ginii, iya, gitu, emote, batu, nii, goo..."
8,Positif,aamiin ya rob mdh an mak trade mark goodby...,"[, aamiin, ya, rob, mdh, an, mak, trade, mark,...","[, aamiin, ya, rob, mdh, an, mak, trade, mark,..."
9,Positif,agar semua berjalan normal kembali goodbyejoko...,"[agar, semua, berjalan, normal, kembali, goodb...","[berjalan, normal, goodbyejokowi, goodbyejokowi]"


In [26]:
# STEMMING
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['Stop_Removal']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['Tweet_Stemmed'] = df['Stop_Removal'].swifter.apply(get_stemmed_term)
print(df['Tweet_Stemmed'])

681
------------------------
 : 
ni : ni
ko : ko
taggar : taggar
goodbyejokowi : goodbyejokowi
sst : sst
biarkan : biar
mahasiswa : mahasiswa
bergerak : gerak
rakyat : rakyat
buruh : buruh
stm : stm
bersatulah : satu
barisan : baris
two : two
period : period
it : it
s : s
been : been
so : so
long : long
i : i
can : can
t : t
wait : wait
anymore : anymore
era : era
jokowi : jokowi
nasakom : nasakom
dihidupkan : hidup
islam : islam
ummat : ummat
disingkirkan : singkir
dakwah : dakwah
ajaran : ajar
dipelintir : pelintir
nusantara : nusantara
ulama : ulama
tokoh : tokoh
dipenjarakan : penjara
revolusi : revolusi
menghacurkan : menghacurkan
rezim : rezim
yg : yg
tirani : tirani
trand : trand
mundurr : mundurr
ngak : ngak
dihiraukan : hirau
tdk : tdk
pura : pura
bangsa : bangsa
tanah : tanah
air : air
oo : oo
ginii : ginii
iya : iya
gitu : gitu
emote : emote
batu : batu
nii : nii
inisialm : inisialm
berkahramadhanditvone : berkahramadhanditvone
aamiin : aamiin
ya : ya
rob : rob
mdh : mdh
an 

kendor : kendor
penipu : tipu
tumbang : tumbang
dibilang : bilang
kegiatannya : giat
model : model
you : you
know : know
gagah : gagah
gimana : gimana
ngiris : ngiris
bawang : bawang
sepanci : panci
ndang : ndang
balio : balio
neng : neng
panggonmu : panggon
selamat : selamat
hape : hape
baiknya : baik
pernyataan : nyata
menuda : menuda
pemilu : milu
undang : undang
selesai : selesai
manis : manis
lbp : lbp
diam : diam
gaduh : gaduh
dgn : dgn
opini : opini
diluar : luar
tugasnya : tugas
wkwkwkw : wkwkwkw
pencitraan : citra
mulu : mulu
lihat : lihat
yaa : yaa
kalo : kalo
sadar : sadar
wkwkwk : wkwkwk
ratusan : ratus
juta : juta
tren : tren
modal : modal
selfi : selfi
priode : priode
maksa : maksa
banget : banget
kebodohan : bodoh
just : just
say : say
semenjak : semenjak
kenyataan : nyata
biidznillah : biidznillah
sendok : sendok
don : don
adam : adam
rahasia : rahasia
masak : masak
dipaksa : paksa
kudeta : kudeta
lha : lha
enak : enak
binatang : binatang
cebong : cebong
ginting : ginti

Pandas Apply:   0%|          | 0/149 [00:00<?, ?it/s]

0                      [, ni, ko, taggar, goodbyejokowi]
1                  [, sst, goodbyejokowi, goodbyejokowi]
2      [, biar, mahasiswa, gerak, rakyat, buruh, stm,...
3      [, two, period, it, s, been, so, long, i, can,...
4      [era, jokowi, nasakom, hidup, islam, ummat, is...
                             ...                        
144    [udah, ngaak, hargain, baik, mundur, lbih, hor...
145                                    [goodbyejokowi, ]
146    [president, gagal, listrik, solar, langka, min...
147    [, suka, ikut, ya, goodbyejokowi, goodbyepresi...
148    [, mohon, maaf, ber, ikrar, tritura, juta, ele...
Name: Tweet_Stemmed, Length: 149, dtype: object


In [27]:
df.to_csv("dataset_preprocessed3.csv")

In [7]:
df = pd.read_csv('dataset_preprocessed.csv')
df

,Unnamed: 0,date,username,tweet,Tokenization,Stop_Removal,Tweet_Stemmed
0,0,2022-04-06 06:59:54,masaan64,bosgenk ada apa ada taggar,"['bosgenk', 'ada', 'apa', 'ada', 'taggar']","['bosgenk', 'taggar']","['bosgenk', 'taggar']"
1,1,2022-04-06 06:59:27,RBugeh,bosgenk sssst,"['bosgenk', 'sssst']","['bosgenk', 'sssst']","['bosgenk', 'sssst']"
2,2,2022-04-06 06:58:56,OmegaF1998,ak jangan biarkan mahasiswa bergerak sendirian...,"['ak', 'jangan', 'biarkan', 'mahasiswa', 'berg...","['ak', 'biarkan', 'mahasiswa', 'bergerak', 'ra...","['ak', 'biar', 'mahasiswa', 'gerak', 'rakyat',..."
3,3,2022-04-06 06:56:40,Aki_jupi,two period been long can wait anymore,"['two', 'period', 'been', 'long', 'can', 'wait...","['two', 'period', 'been', 'long', 'can', 'wait...","['two', 'period', 'been', 'long', 'can', 'wait..."
4,4,2022-04-06 06:56:37,mhbmuhib17,era jokowi nasakom dihidupkan kembali islam da...,"['era', 'jokowi', 'nasakom', 'dihidupkan', 'ke...","['era', 'jokowi', 'nasakom', 'dihidupkan', 'is...","['era', 'jokowi', 'nasakom', 'hidup', 'islam',..."
...,...,...,...,...,...,...,...
199,199,2022-04-06 05:14:34,JakTar9,president gagal listrik naik solar langka miny...,"['president', 'gagal', 'listrik', 'naik', 'sol...","['president', 'gagal', 'listrik', 'solar', 'la...","['president', 'gagal', 'listrik', 'solar', 'la..."
200,200,2022-04-06 05:14:31,Hartati44247904,suka ikutan,"['suka', 'ikutan']","['suka', 'ikutan']","['suka', 'ikut']"
201,201,2022-04-06 05:14:28,PopoleTv,NaN,[''],[''],['']
202,202,2022-04-06 05:14:19,SaskiOktavia,nku,['nku'],['nku'],['nku']


In [43]:
df1 = df['tweet']

In [45]:
df1.head()

0                           bosgenk ada apa ada taggar
1                                        bosgenk sssst
2    ak jangan biarkan mahasiswa bergerak sendirian...
3                two period been long can wait anymore
4    era jokowi nasakom dihidupkan kembali islam da...
Name: tweet, dtype: object

In [47]:
df1.to_csv("dataset_untuk_label.csv", index=False)

In [50]:
df1.dropna()

0                             bosgenk ada apa ada taggar
1                                          bosgenk sssst
2      ak jangan biarkan mahasiswa bergerak sendirian...
3                  two period been long can wait anymore
4      era jokowi nasakom dihidupkan kembali islam da...
                             ...                        
198                                             gerindra
199    president gagal listrik naik solar langka miny...
200                                          suka ikutan
202                                                  nku
203    mohon maaf kita dulu ber ikrar tritura jutaan ...
Name: tweet, Length: 165, dtype: object